In [1]:
import os, fnmatch, cv2
import numpy as np
import matplotlib.pyplot as plt
from imageio import imread
from register import alignImages
from skimage.measure import compare_ssim

from models import UNet2D
os.environ['CUDA_VISIBLE_DEVICES']='3'

W0127 14:58:06.981426 140005688190720 deprecation.py:506] From /home/grads/e/ethanycx/anaconda3/envs/ame/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [2]:
data_path = 'Paired SEM images/'
fnames = os.listdir(data_path)
hr_fnames = sorted([data_path+fname for fname in fnames if fnmatch.fnmatch(fname, '*20000.bmp')])
lr_fnames = sorted([data_path+fname for fname in fnames if fnmatch.fnmatch(fname, '*10000.bmp')])

hr_lst = [imread(f)[:944] for f in hr_fnames]
lr_lst = [imread(f)[:944] for f in lr_fnames]
lr_reg_lst = [alignImages(lr_lst[idx], hr_lst[idx])[0] for idx in range(len(hr_lst))]

In [3]:
model = UNet2D('models/', 'pooled_batch8_lib64_kshape')
sources = np.array(lr_reg_lst)[:22,:,:960][...,None]/255.0
targets = np.array(hr_lst)[:22,:,:960][...,None]/255.0

test_X = np.vstack([np.array(lr_reg_lst)[:22,:312, 960:], np.array(lr_reg_lst)[:22,312:624, 960:], np.array(lr_reg_lst)[:22,624:936, 960:]])/255.0
test_Y = np.vstack([np.array(hr_lst)[:22,:312, 960:], np.array(hr_lst)[:22,312:624, 960:], np.array(hr_lst)[:22,624:936, 960:]])/255.0

# model.train(sources, targets, [256, 256], validation = None, steps = 120000, batch_size = 8, seed = 1)

In [4]:
def evaluate_psnr_wh(idx, getimg=False, size=256, save_to=None):
    model = UNet2D('models/', 'pooled_batch8_lib64_kshape')
    test_X = np.stack([lr_reg_lst[idx][:314, 960:], lr_reg_lst[idx][314:628, 960:], lr_reg_lst[idx][628:942, 960:]])
    test_Y = np.stack([hr_lst[idx][:314, 960:], hr_lst[idx][314:628, 960:], hr_lst[idx][628:942, 960:]])
    
    if size is None:
        pred = model.predict(np.array(lr_reg_lst[idx:idx+1]).astype('float32')/255.0)[0]
    else:
        pred = model.crop_predict(np.array(lr_reg_lst[idx]).astype('float32')/255.0, size=size, margin=32)
    pred = np.stack([pred[:314, 960:], pred[314:628, 960:], pred[628:942, 960:]])
    input_psnrs = np.array([cv2.PSNR(x, y) for x, y in zip(test_X, test_Y)])
    pred_psnrs = np.array([cv2.PSNR(np.clip(x*255, 0, 255), y.astype('float32')) for x, y in zip(pred, test_Y)])
    
    input_ssims = np.array([compare_ssim(x, y, data_range=255.0) for x, y in zip(test_X, test_Y)])
    pred_ssims = np.array([compare_ssim(np.clip(x*255, 0, 255), y.astype('float32'), data_range=255.0) for x, y in zip(pred, test_Y)])
    if getimg:
        print(-input_psnrs+pred_psnrs)
        return pred
    if save_to:
        save_array = np.stack([test_X, test_Y, pred])
        np.save(save_to, save_array)

    return input_psnrs, pred_psnrs, input_ssims, pred_ssims

In [6]:
def evaluate_mean_wh(size, save_to=None):
    if save_to:
        mip, mpp, mis, mps = np.stack([evaluate_psnr_wh(idx,size=size, save_to=save_to+'%d.npy'%idx) for idx in range(22)]).mean((0, -1))
#     else:
    return mpp-mip, mps-mis

In [10]:
# batch8_lib64_kshape
print(evaluate_mean_wh(None))
print(evaluate_mean_wh(512))
print(evaluate_mean_wh(384))
print(evaluate_mean_wh(256))
print(evaluate_mean_wh(128))

(2.0986001093646856, 0.08660230379886646)
(1.9921139749495325, 0.0867660098926189)
(1.8940178657562967, 0.0860400328388457)
(1.8401389625750575, 0.08515587293785803)
(1.302734046356946, 0.07973252207721482)


In [ ]:
# batch8_lib32_kshape
print(evaluate_mean_wh(None))
print(evaluate_mean_wh(512))
print(evaluate_mean_wh(384))
print(evaluate_mean_wh(256))
print(evaluate_mean_wh(128))

W0727 13:07:42.158524 139913352476416 deprecation_wrapper.py:119] From /home/grads/e/ethanycx/nb_working_space/SEM/models.py:143: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0727 13:07:42.160198 139913352476416 deprecation_wrapper.py:119] From /home/grads/e/ethanycx/nb_working_space/SEM/models.py:143: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



(2.015785924642465, 0.08492159461281923)
(1.9880621897474633, 0.08548796755541299)
(1.9423706305880444, 0.08534492186031684)
(1.796649630084918, 0.08428913259481807)


In [6]:
# batch8_lib16_kshape
print(evaluate_mean_wh(None))
print(evaluate_mean_wh(512))
print(evaluate_mean_wh(384))
print(evaluate_mean_wh(256))
print(evaluate_mean_wh(128))

W0728 01:35:03.791324 140661837297408 deprecation_wrapper.py:119] From /home/grads/e/ethanycx/nb_working_space/SEM/models.py:143: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0728 01:35:03.792351 140661837297408 deprecation_wrapper.py:119] From /home/grads/e/ethanycx/nb_working_space/SEM/models.py:143: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



(2.0170695768533804, 0.08527970102854315)
(1.9834225485905215, 0.08628199959832783)
(1.8570835894125999, 0.08540288425241077)
(1.6898606510306031, 0.08332093648147659)
(1.297795669278134, 0.07987768364141379)
